In [1]:
import numpy as np
import pandas as pd

import time
import pandas as pd 
import ast

# from visualisation import generate_map

c_df = pd.read_csv('data/combined_dataframes.csv')
c_df.tail()

,latitude,longitude,bright_ti4,acq_date,instrument,confidence,bright_ti5
708003,-10.16375,142.13250,[333.5],['2019-12-13'],['VIIRS'],['n'],[295.5]
708004,-10.16375,142.14375,[340.0],['2019-12-23'],['VIIRS'],['n'],[296.6]
708005,-10.16375,142.14750,"[341.7, 309.7]",['2019-12-23'],['VIIRS'],['n'],"[296.4, 274.4]"
708006,-10.09625,142.16250,[315.5],['2019-10-08'],['VIIRS'],['n'],[290.3]
708007,-9.39875,142.64625,"[367.0, 345.1]",['2019-12-02'],['VIIRS'],"['h', 'n']","[301.8, 301.9]"


In [2]:
# Convert the contents of the pandas array from strings looking like lists to actual lists
brightness_MODIS = c_df.loc[:,'bright_ti4'].apply(ast.literal_eval)
brightness_VIIRS = c_df.loc[:,'bright_ti5'].apply(ast.literal_eval)
instrument = c_df.loc[:,'instrument'].apply(ast.literal_eval)

In [3]:
risk = np.zeros(len(c_df.latitude))
risk

array([0., 0., 0., ..., 0., 0., 0.])

In [4]:
# calculate the mean of each of the brightnesses
import statistics 
for i,list in enumerate(brightness_MODIS):
    risk[i] += statistics.mean(list)

In [5]:
for i,list in enumerate(brightness_VIIRS):
    risk[i] += statistics.mean(list)

In [6]:
# calculate the average brightness
for i,list in enumerate(risk):
    risk[i] = risk[i] / len(instrument[i]) # divide by the number of instruments i.e. mean of 1 or mean of 2
# risk = np.mean(brightness_MODIS, brightness_VIIRS) # does not work

In [7]:
#other stuff I tried
#  ast.literal_eval(c_df.bright_ti4[1])
# c_df[2:]= c_df['bright_ti4'].apply(ast.literal_eval)
# c_df['Risk']= c_df['bright_ti4'].apply(len)
# c_df['Risk']=np.average(c_df.bright_ti4[1:20]) + np.average(c_df.bright_ti5[1:20])
# c_df.head()
# np.average(c_df.bright_ti4[1:20])
# c_df.bright_ti4.aggregate(sum)
# sum(axis=1)
# c_df

In [8]:
# time = dict()
# for i in range(len(risk)):
#     if c_df.loc[i]["acq_date"] in time:
#         time[c_df.loc[i]["acq_date"]] += 1
#     else:
#         time[c_df.loc[i]["acq_date"]] = 1

In [9]:
import datetime
import time
# create function to calculate the difference between two days
def calculateDays(date1, date2):
    day1 = time.strptime(date1, "%Y-%m-%d")
    day1 = datetime.datetime(day1[0], day1[1], day1[2])
    day2 = time.strptime(date2, "%Y-%m-%d")
    day2 = datetime.datetime(day2[0], day2[1], day2[2])
    #today = datetime.datetime.today()
    interval = day1 - day2
    return interval.days

In [10]:
# create a numpy array to store the result of timeRange
timeRange = np.zeros(len(c_df.latitude))

In [11]:
timeData = c_df["acq_date"].apply(ast.literal_eval)
for i, value in enumerate(timeData):
    # if only one day, the result will be the difference between that and the date today
    if len(value) == 1:
        timeRange[i] = abs(calculateDays("2020-02-15",timeData[i][0]))
    # if more than one day, the result will be the difference between the start day and the end day
    elif len(value) > 1:
        # start day
        date1 = timeData[i][0]
        # end day
        date2 = timeData[i][-1]
        timeRange[i] = abs(calculateDays(date2,date1))

In [12]:
# divided by the time range
for i,list in enumerate(risk):
    risk[i] = risk[i] / timeRange[i]

In [13]:
risk

array([16.63513514, 14.38181818, 14.25227273, ..., 11.31666667,
        4.66      ,  8.772     ])

In [14]:
c_df['Risk'] = risk
c_df.iloc[0:50]

,latitude,longitude,bright_ti4,acq_date,instrument,confidence,bright_ti5,Risk
0,-43.11875,146.61375,[313.6],['2020-01-09'],['MODIS'],[68],[301.9],16.635135
1,-42.83375,147.20250,[328.7],['2020-01-02'],['MODIS'],[85],[304.1],14.381818
2,-42.83000,147.21750,[324.9],['2020-01-02'],['MODIS'],[75],[302.2],14.252273
3,-42.60125,147.00375,[323.5],['2020-01-02'],['MODIS'],[66],[295.9],14.077273
4,-42.59375,146.98500,[322.1],['2020-01-03'],['MODIS'],[62],[297.0],14.397674
5,-42.59375,146.99625,[336.7],['2020-01-02'],['MODIS'],[83],[302.9],14.536364
6,-42.58250,147.01875,[338.7],['2020-01-02'],['MODIS'],[80],[302.2],14.565909
7,-42.57875,147.03000,[350.5],['2020-01-02'],['MODIS'],[96],[300.3],14.790909
8,-41.74625,148.00875,[340.1],['2020-01-09'],['MODIS'],[89],[299.6],17.289189
9,-41.74625,148.02375,[319.7],['2020-01-09'],['MODIS'],[37],[297.6],16.683784
